In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns

## Read datasets and extract common features

In [2]:
df1  = pd.read_csv('aqardata_2.csv')

In [3]:
df1.head()

,mainlocation,sublocation,neighborhood,frontage,purpose,streetwidth,size,Pricepm
0,الرياض,غرب الرياض,حي ظهرة لبن,شمال,سكني,20.0,727,1800
1,بريدة,NaN,حي مشعل,غرب,NaN,15.0,450,950
2,الخبر,NaN,حي الحمرا,غرب,تجاري,100.0,1450,3500
3,الخبر,NaN,حي الحزام الاخضر,شرق,NaN,15.0,440,2700
4,بريدة,NaN,حي الرحاب,جنوب غربي,NaN,40.0,784,950


## First clean and pre-process first dataframe (df1) and keep only important features

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mainlocation  2951 non-null   object 
 1   sublocation   2276 non-null   object 
 2   neighborhood  2951 non-null   object 
 3   frontage      2951 non-null   object 
 4   purpose       2604 non-null   object 
 5   streetwidth   2948 non-null   float64
 6   size          2951 non-null   int64  
 7   Pricepm       2951 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 184.6+ KB


In [5]:
# We want a new column with whole price instead of Pricepm
df1['Price'] = df1['size'] * df1['Pricepm']
df1.drop(columns = 'Pricepm' , inplace = True)

In [6]:
df1.head()

,mainlocation,sublocation,neighborhood,frontage,purpose,streetwidth,size,Price
0,الرياض,غرب الرياض,حي ظهرة لبن,شمال,سكني,20.0,727,1308600
1,بريدة,NaN,حي مشعل,غرب,NaN,15.0,450,427500
2,الخبر,NaN,حي الحمرا,غرب,تجاري,100.0,1450,5075000
3,الخبر,NaN,حي الحزام الاخضر,شرق,NaN,15.0,440,1188000
4,بريدة,NaN,حي الرحاب,جنوب غربي,NaN,40.0,784,744800


In [7]:
# Drop streetwidth, frontage and sublocation although these three features is important in deciding the price of the land but in our case we want to concatenate this dataset with other datasets and the others don't have these two features
df1.drop(columns = ['frontage' , 'streetwidth' , 'sublocation'] , inplace = True)

In [8]:
df1.head()

,mainlocation,neighborhood,purpose,size,Price
0,الرياض,حي ظهرة لبن,سكني,727,1308600
1,بريدة,حي مشعل,NaN,450,427500
2,الخبر,حي الحمرا,تجاري,1450,5075000
3,الخبر,حي الحزام الاخضر,NaN,440,1188000
4,بريدة,حي الرحاب,NaN,784,744800


In [9]:
# Before finishing from this dataset we will add one extra column that will represent the region of the land(منطقة)
df1['mainlocation'].unique()

array(['الرياض', 'بريدة', 'الخبر', 'جدة', 'سكاكا', 'الجموم',
       'المدينة المنورة', 'الدمام', 'مكة المكرمة', 'عرعر', 'عنيزة',
       'خميس مشيط', 'الدرعية', 'المزاحمية', 'الزلفي', 'جازان', 'الدلم',
       'ثول', 'الخرج', 'الرس', 'رابغ', 'الهفوف', 'احد رفيده', 'ابها',
       'صامطة', 'ثادق', 'الهياثم', 'الظهران', 'القطيف', 'الطائف', 'بيش',
       'صبيا', 'ابو عريش', 'مدينة الملك عبدالله الاقتصادية', 'رماح',
       'تبوك', 'نجران', 'المجمعة', 'الشنان', 'حائل', 'رياض الخبراء',
       'النبهانية', 'صفوى', 'تاروت', 'المذنب'], dtype=object)

In [10]:
df1.insert(0 , 'region' , None)

In [11]:
# As i am from saudi arabia i know the sublocation of these
riyadh_cities = ['الرياض', 'الدرعية', 'المزاحمية', 'الزلفي', 'الدلم', 'رماح', 'الهياثم' , 'المجمعة','ثادق']

qassim_cities = ['بريدة', 'عنيزة', 'الخرج', 'الرس', 'النبهانية', 'رياض الخبراء', 'المذنب']

eastern_region_cities = ['الخبر', 'الدمام', 'الهفوف', 'الظهران', 'القطيف', 'صفوى', 'تاروت']

makkah_cities = ['جدة', 'الجموم', 'مكة المكرمة', 'رابغ', 'الطائف', 'ثول' , 'مدينة الملك عبدالله الاقتصادية']

aljouf_cities = ['سكاكا']

madinah_cities = ['المدينة المنورة']

northern_frontier_cities = ['عرعر']

assir_cities = ['خميس مشيط', 'ابها', 'احد رفيده']

jazan_cities = ['جازان', 'بيش', 'صبيا', 'ابو عريش', 'صامطة']

tabuk_cities = ['تبوك']

najran_cities = ['نجران']

hail_cities = ['حائل' , 'الشنان']

df1.loc[df1["mainlocation"].isin(riyadh_cities) ,"region"]="منطقة الرياض"
df1.loc[df1["mainlocation"].isin(qassim_cities) ,"region"]="منطقة القصيم"
df1.loc[df1["mainlocation"].isin(eastern_region_cities) ,"region"]="المنطقة الشرقية"
df1.loc[df1["mainlocation"].isin(makkah_cities) ,"region"]="منطقة مكة المكرمة"
df1.loc[df1["mainlocation"].isin(aljouf_cities) ,"region"]="منطقة الجوف"
df1.loc[df1["mainlocation"].isin(madinah_cities) ,"region"]="منطقة المدينة المنورة"
df1.loc[df1["mainlocation"].isin(northern_frontier_cities) ,"region"]="منطقة الحدود الشمالية"
df1.loc[df1["mainlocation"].isin(jazan_cities) ,"region"]="منطقة جازان"
df1.loc[df1["mainlocation"].isin(tabuk_cities) ,"region"]="منطقة تبوك"
df1.loc[df1["mainlocation"].isin(najran_cities) ,"region"]="منطقة نجران"
df1.loc[df1["mainlocation"].isin(hail_cities) ,"region"]="منطقة حائل"
df1.loc[df1["mainlocation"].isin(assir_cities) ,"region"]="منطقة عسير"


In [12]:
df1.head()

,region,mainlocation,neighborhood,purpose,size,Price
0,منطقة الرياض,الرياض,حي ظهرة لبن,سكني,727,1308600
1,منطقة القصيم,بريدة,حي مشعل,NaN,450,427500
2,المنطقة الشرقية,الخبر,حي الحمرا,تجاري,1450,5075000
3,المنطقة الشرقية,الخبر,حي الحزام الاخضر,NaN,440,1188000
4,منطقة القصيم,بريدة,حي الرحاب,NaN,784,744800


In [13]:
# Until this point we are supposed to be finished from this dataset, but one last point is to deal with null values.
# Although it will be a good practice to leave it until we do the concatenat of different  dataframes together to do it once. But in our case this small dataset is the only dataset that is dirty and I will fix only one null column.
df1.isna().sum()

region            0
mainlocation      0
neighborhood      0
purpose         347
size              0
Price             0
dtype: int64

In [14]:
df1['purpose'].value_counts(dropna=False)

سكني             2005
NaN               347
تجاري             329
سكني أو تجاري     268
غير محدد            2
Name: purpose, dtype: int64

In [15]:
# I chose to replace NaN with  غير محدد since I see both of them fall under the same meaning
df1['purpose'].fillna('غير محدد',inplace = True)

In [16]:
df1.isna().sum()

region          0
mainlocation    0
neighborhood    0
purpose         0
size            0
Price           0
dtype: int64

In [17]:
# One last step is to add an empty column that serves as a date
df1['اليوم'] = None
df1['الشهر'] = None
df1['السنة'] = None
df1['نوع العقار'] = None
df1['عدد العقارات'] = None

In [18]:
df1.head()

,region,mainlocation,neighborhood,purpose,size,Price,اليوم,الشهر,السنة,نوع العقار,عدد العقارات
0,منطقة الرياض,الرياض,حي ظهرة لبن,سكني,727,1308600,None,None,None,None,None
1,منطقة القصيم,بريدة,حي مشعل,غير محدد,450,427500,None,None,None,None,None
2,المنطقة الشرقية,الخبر,حي الحمرا,تجاري,1450,5075000,None,None,None,None,None
3,المنطقة الشرقية,الخبر,حي الحزام الاخضر,غير محدد,440,1188000,None,None,None,None,None
4,منطقة القصيم,بريدة,حي الرحاب,غير محدد,784,744800,None,None,None,None,None


In [19]:
# Final step is to  rearrange columns so it will be ready to concatenate
purpose = df1.pop('purpose')
price = df1.pop('Price')
size = df1.pop('size')
df1.insert(6 ,'purpose' ,purpose)
df1.insert(9 , 'size',size)
df1.insert(10 , 'price',price)
df1.head()

,region,mainlocation,neighborhood,اليوم,الشهر,السنة,purpose,نوع العقار,عدد العقارات,size,price
0,منطقة الرياض,الرياض,حي ظهرة لبن,None,None,None,سكني,None,None,727,1308600
1,منطقة القصيم,بريدة,حي مشعل,None,None,None,غير محدد,None,None,450,427500
2,المنطقة الشرقية,الخبر,حي الحمرا,None,None,None,تجاري,None,None,1450,5075000
3,المنطقة الشرقية,الخبر,حي الحزام الاخضر,None,None,None,غير محدد,None,None,440,1188000
4,منطقة القصيم,بريدة,حي الرحاب,None,None,None,غير محدد,None,None,784,744800


In [20]:
df1.shape

(2951, 11)

## Clean and pre-process second dataframe (df2)




In [21]:
df2  = pd.read_csv('realestate_2023_q1.csv')

In [22]:
df2.head()

,رقم مرجعي,المنطقة,المدينة,الحي,المخطط,رقم القطعة,التاريخ,تصنيف العقار,نوع العقار,عدد العقارات,السعر بالريال السعودي,المساحة,سعر المتر المربع
0,18923186,منطقة الرياض,الافلاج,حي/أخرى,مخطط/أخرى,قطعة 505,1/1/2023,سكني,قطعة أرض,1,32000,630.0,50.7936
1,18920819,منطقة الرياض,الافلاج,حي/الريان,مخطط/872,قطعة 1500,1/1/2023,سكني,قطعة أرض,1,60000,690.0,86.9565
2,18923523,منطقة الرياض,الخرج,حي/827 حى هجرة الصاع,مخطط/أخرى,قطعة 948,1/1/2023,سكني,قطعة أرض,1,20000,630.0,31.7460
3,18909006,منطقة الرياض,الخرج,حي/الاندلس,مخطط/أخرى,قطعة 555,1/1/2023,سكني,قطعة أرض,1,140000,333.0,420.4204
4,18857540,منطقة الرياض,الخرج,حي/الخالديه,مخطط/97,قطعة 182,1/1/2023,سكني,قطعة أرض,1,50000,380.0,131.5789


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44091 entries, 0 to 44090
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   رقم مرجعي              44091 non-null  int64  
 1   المنطقة                44091 non-null  object 
 2   المدينة                44091 non-null  object 
 3   الحي                   44091 non-null  object 
 4   المخطط                 44091 non-null  object 
 5   رقم القطعة             44091 non-null  object 
 6   التاريخ                44091 non-null  object 
 7   تصنيف العقار           44091 non-null  object 
 8   نوع العقار             44091 non-null  object 
 9   عدد العقارات           44091 non-null  int64  
 10  السعر بالريال السعودي  44091 non-null  int64  
 11  المساحة                44091 non-null  float64
 12  سعر المتر المربع       44091 non-null  float64
dtypes: float64(2), int64(3), object(8)
memory usage: 4.4+ MB


In [24]:
df2.isna().sum()

رقم مرجعي                0
المنطقة                  0
المدينة                  0
الحي                     0
المخطط                   0
رقم القطعة               0
التاريخ                  0
تصنيف العقار             0
نوع العقار               0
عدد العقارات             0
السعر بالريال السعودي    0
المساحة                  0
سعر المتر المربع         0
dtype: int64

In [25]:
# Drop less important columns
df2.drop(columns = ['رقم مرجعي' , 'المخطط' , 'رقم القطعة' , 'سعر المتر المربع'] , inplace = True)

In [26]:
df2.head()

,المنطقة,المدينة,الحي,التاريخ,تصنيف العقار,نوع العقار,عدد العقارات,السعر بالريال السعودي,المساحة
0,منطقة الرياض,الافلاج,حي/أخرى,1/1/2023,سكني,قطعة أرض,1,32000,630.0
1,منطقة الرياض,الافلاج,حي/الريان,1/1/2023,سكني,قطعة أرض,1,60000,690.0
2,منطقة الرياض,الخرج,حي/827 حى هجرة الصاع,1/1/2023,سكني,قطعة أرض,1,20000,630.0
3,منطقة الرياض,الخرج,حي/الاندلس,1/1/2023,سكني,قطعة أرض,1,140000,333.0
4,منطقة الرياض,الخرج,حي/الخالديه,1/1/2023,سكني,قطعة أرض,1,50000,380.0


In [27]:
# First let's deal with الحي column since it's not clean
df2['الحي'] = df2['الحي'].str.replace('حي/', '')

In [28]:
df2['الحي'].value_counts()

أخرى                     2913
الخير                     902
البرود                    630
جلمودة                    615
العزيزية                  570
                         ... 
وادى الجناح                 1
العوشزية                    1
العلياء                     1
ق/ ر/ 5/ ت/061              1
28/ 1425/ ع/ 6 المعدل       1
Name: الحي, Length: 4091, dtype: int64

In [29]:
# Let's add three column that represents the date
df2['التاريخ'] = pd.to_datetime(df2['التاريخ'])
df2.insert(3,'اليوم' , df2['التاريخ'].dt.day)
df2.insert(4,'الشهر' , df2['التاريخ'].dt.month)
df2.insert(5,'السنة' , df2['التاريخ'].dt.year)
df2.drop(columns = 'التاريخ',inplace = True)

In [30]:
# Final step is to  rearrange columns so it will be ready to concatenate
size = df2.pop('المساحة ')
price = df2.pop('السعر بالريال السعودي')
df2.insert(9 , 'المساحة' , size)
df2.insert(10 , 'السعر بالريال السعودي' , price)

In [31]:
df2.head()

,المنطقة,المدينة,الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,المساحة,السعر بالريال السعودي
0,منطقة الرياض,الافلاج,أخرى,1,1,2023,سكني,قطعة أرض,1,630.0,32000
1,منطقة الرياض,الافلاج,الريان,1,1,2023,سكني,قطعة أرض,1,690.0,60000
2,منطقة الرياض,الخرج,827 حى هجرة الصاع,1,1,2023,سكني,قطعة أرض,1,630.0,20000
3,منطقة الرياض,الخرج,الاندلس,1,1,2023,سكني,قطعة أرض,1,333.0,140000
4,منطقة الرياض,الخرج,الخالديه,1,1,2023,سكني,قطعة أرض,1,380.0,50000


In [32]:
df2.shape

(44091, 11)

## Clean and pre-process third dataframe (df3)


In [33]:
df3  = pd.read_csv('docrealestatesale_2023_q2.csv')

In [34]:
df3.head()

,المنطقة,المدينة,المدينة / الحي,الرقم المرجعي للصفقة,تاريخ الصفقة ميلادي,تاريخ الصفقة هجري,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الرياض,الرياض/السويدى,19683341,2023/04/01,1444/09/10,تجاري,قطعة أرض,2,2500000.0,1000.00
1,منطقة الرياض,الرياض,الرياض/المهدية,19682474,2023/04/01,1444/09/10,سكني,قطعة أرض,1,1400000.0,270.00
2,منطقة مكة المكرمه,الطائف,الطائف/ الحلقة,19683061,2023/04/01,1444/09/10,سكني,قطعة أرض,1,1200000.0,1096.12
3,منطقة الرياض,الرياض,الرياض/الزهرة,19683218,2023/04/01,1444/09/10,تجاري,قطعة أرض,1,1000000.0,625.00
4,منطقة الشرقية,قرية العليا,قرية العليا/أخرى,19620608,2023/04/01,1444/09/10,سكني,قطعة أرض,1,20000.0,920.00


In [35]:
# Drop less important columns
df3.drop(columns = ['الرقم المرجعي للصفقة' , 'تاريخ الصفقة هجري'] , inplace = True)
df3.head()

,المنطقة,المدينة,المدينة / الحي,تاريخ الصفقة ميلادي,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الرياض,الرياض/السويدى,2023/04/01,تجاري,قطعة أرض,2,2500000.0,1000.00
1,منطقة الرياض,الرياض,الرياض/المهدية,2023/04/01,سكني,قطعة أرض,1,1400000.0,270.00
2,منطقة مكة المكرمه,الطائف,الطائف/ الحلقة,2023/04/01,سكني,قطعة أرض,1,1200000.0,1096.12
3,منطقة الرياض,الرياض,الرياض/الزهرة,2023/04/01,تجاري,قطعة أرض,1,1000000.0,625.00
4,منطقة الشرقية,قرية العليا,قرية العليا/أخرى,2023/04/01,سكني,قطعة أرض,1,20000.0,920.00


In [36]:
# Store only the neighbor
df3['المدينة / الحي'] = df3['المدينة / الحي'].str.split('/').str[1]

In [37]:
# Let's add three column that represents the date
df3['تاريخ الصفقة ميلادي'] = pd.to_datetime(df3['تاريخ الصفقة ميلادي'])
df3.insert(3,'اليوم' , df3['تاريخ الصفقة ميلادي'].dt.day)
df3.insert(4,'الشهر' , df3['تاريخ الصفقة ميلادي'].dt.month)
df3.insert(5,'السنة' , df3['تاريخ الصفقة ميلادي'].dt.year)
df3.drop(columns = 'تاريخ الصفقة ميلادي',inplace = True)

In [38]:
df3.head()

,المنطقة,المدينة,المدينة / الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الرياض,السويدى,1,4,2023,تجاري,قطعة أرض,2,2500000.0,1000.00
1,منطقة الرياض,الرياض,المهدية,1,4,2023,سكني,قطعة أرض,1,1400000.0,270.00
2,منطقة مكة المكرمه,الطائف,الحلقة,1,4,2023,سكني,قطعة أرض,1,1200000.0,1096.12
3,منطقة الرياض,الرياض,الزهرة,1,4,2023,تجاري,قطعة أرض,1,1000000.0,625.00
4,منطقة الشرقية,قرية العليا,أخرى,1,4,2023,سكني,قطعة أرض,1,20000.0,920.00


In [39]:
# Final step is to  rearrange columns so it will be ready to concatenate
size = df3.pop('المساحة')
price = df3.pop('السعر')
df3.insert(9 , 'المساحة' , size)
df3.insert(10 , 'السعر' , price)

In [40]:
df3.head()

,المنطقة,المدينة,المدينة / الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,المساحة,السعر
0,منطقة الرياض,الرياض,السويدى,1,4,2023,تجاري,قطعة أرض,2,1000.00,2500000.0
1,منطقة الرياض,الرياض,المهدية,1,4,2023,سكني,قطعة أرض,1,270.00,1400000.0
2,منطقة مكة المكرمه,الطائف,الحلقة,1,4,2023,سكني,قطعة أرض,1,1096.12,1200000.0
3,منطقة الرياض,الرياض,الزهرة,1,4,2023,تجاري,قطعة أرض,1,625.00,1000000.0
4,منطقة الشرقية,قرية العليا,أخرى,1,4,2023,سكني,قطعة أرض,1,920.00,20000.0


In [41]:
df3.shape

(38811, 11)

 ## Clean and pre-process last dataframe (df4)

In [42]:
df4  = pd.read_csv('docrealestatesale_2023_q3.csv')

In [43]:
df4.head()

,المنطقة,المدينة,المدينة / الحي,الرقم المرجعي للصفقة,تاريخ الصفقة ميلادي,تاريخ الصفقة هجري,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الافلاج,الافلاج/ الملك عبدالله,20289286,2023/07/01,1444/12/13,سكني,قطعة أرض,1,53000.0,660.00
1,منطقة الرياض,الرياض,الرياض/عريض,20289169,2023/07/01,1444/12/13,سكني,قطعة أرض,1,300000.0,450.00
2,منطقة الرياض,الرياض,الرياض/نمار,20289165,2023/07/01,1444/12/13,سكني,قطعة أرض,1,430000.0,300.00
3,منطقة المدينة المنوره,الصويدره,الصويدره/أخرى,20289168,2023/07/01,1444/12/13,سكني,قطعة أرض,1,73000.0,642.06
4,منطقة حائل,الشنان,الشنان/أخرى,20289133,2023/07/01,1444/12/13,سكني,قطعة أرض,1,50000.0,1050.00


In [44]:
# Similiar pre-processing as the third dataset

# Drop less important columns
df4.drop(columns = ['الرقم المرجعي للصفقة' , 'تاريخ الصفقة هجري'] , inplace = True)
df4.head()

,المنطقة,المدينة,المدينة / الحي,تاريخ الصفقة ميلادي,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الافلاج,الافلاج/ الملك عبدالله,2023/07/01,سكني,قطعة أرض,1,53000.0,660.00
1,منطقة الرياض,الرياض,الرياض/عريض,2023/07/01,سكني,قطعة أرض,1,300000.0,450.00
2,منطقة الرياض,الرياض,الرياض/نمار,2023/07/01,سكني,قطعة أرض,1,430000.0,300.00
3,منطقة المدينة المنوره,الصويدره,الصويدره/أخرى,2023/07/01,سكني,قطعة أرض,1,73000.0,642.06
4,منطقة حائل,الشنان,الشنان/أخرى,2023/07/01,سكني,قطعة أرض,1,50000.0,1050.00


In [45]:
# Store only the neighbor
df4['المدينة / الحي'] = df4['المدينة / الحي'].str.split('/').str[1]

In [46]:
# Let's add three column that represents the date
df4['تاريخ الصفقة ميلادي'] = pd.to_datetime(df4['تاريخ الصفقة ميلادي'])
df4.insert(3,'اليوم' , df4['تاريخ الصفقة ميلادي'].dt.day)
df4.insert(4,'الشهر' , df4['تاريخ الصفقة ميلادي'].dt.month)
df4.insert(5,'السنة' , df4['تاريخ الصفقة ميلادي'].dt.year)
df4.drop(columns = 'تاريخ الصفقة ميلادي',inplace = True)

In [47]:
df4.head()

,المنطقة,المدينة,المدينة / الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,السعر,المساحة
0,منطقة الرياض,الافلاج,الملك عبدالله,1,7,2023,سكني,قطعة أرض,1,53000.0,660.00
1,منطقة الرياض,الرياض,عريض,1,7,2023,سكني,قطعة أرض,1,300000.0,450.00
2,منطقة الرياض,الرياض,نمار,1,7,2023,سكني,قطعة أرض,1,430000.0,300.00
3,منطقة المدينة المنوره,الصويدره,أخرى,1,7,2023,سكني,قطعة أرض,1,73000.0,642.06
4,منطقة حائل,الشنان,أخرى,1,7,2023,سكني,قطعة أرض,1,50000.0,1050.00


In [48]:
# Final step is to  rearrange columns so it will be ready to concatenate
size = df4.pop('المساحة')
price = df4.pop('السعر')
df4.insert(9 , 'المساحة' , size)
df4.insert(10 , 'السعر' , price)

In [49]:
df4.head()

,المنطقة,المدينة,المدينة / الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,المساحة,السعر
0,منطقة الرياض,الافلاج,الملك عبدالله,1,7,2023,سكني,قطعة أرض,1,660.00,53000.0
1,منطقة الرياض,الرياض,عريض,1,7,2023,سكني,قطعة أرض,1,450.00,300000.0
2,منطقة الرياض,الرياض,نمار,1,7,2023,سكني,قطعة أرض,1,300.00,430000.0
3,منطقة المدينة المنوره,الصويدره,أخرى,1,7,2023,سكني,قطعة أرض,1,642.06,73000.0
4,منطقة حائل,الشنان,أخرى,1,7,2023,سكني,قطعة أرض,1,1050.00,50000.0


In [50]:
df4.shape

(48545, 11)

## After pre-processing each dataframe individually we will concatinate them know

In [52]:
columns = ['المنطقة' , 'المدينة' , 'الحي' , 'اليوم' , 'الشهر' , 'السنة' , 'تصنيف العقار' , 'نوع العقار' , 'عدد العقارات' , 'المساحة' , 'السعر']

final_df = pd.DataFrame(np.concatenate((df4.values , df3.values , df2.values , df1.values) , axis = 0))
final_df.columns = columns


In [53]:
# Finally we have the dataset
final_df

,المنطقة,المدينة,الحي,اليوم,الشهر,السنة,تصنيف العقار,نوع العقار,عدد العقارات,المساحة,السعر
0,منطقة الرياض,الافلاج,الملك عبدالله,1,7,2023,سكني,قطعة أرض,1,660.0,53000.0
1,منطقة الرياض,الرياض,عريض,1,7,2023,سكني,قطعة أرض,1,450.0,300000.0
2,منطقة الرياض,الرياض,نمار,1,7,2023,سكني,قطعة أرض,1,300.0,430000.0
3,منطقة المدينة المنوره,الصويدره,أخرى,1,7,2023,سكني,قطعة أرض,1,642.06,73000.0
4,منطقة حائل,الشنان,أخرى,1,7,2023,سكني,قطعة أرض,1,1050.0,50000.0
...,...,...,...,...,...,...,...,...,...,...,...
134393,منطقة الرياض,الرياض,حي طيبة,None,None,None,سكني,None,None,354,424800
134394,منطقة القصيم,رياض الخبراء,حي النزهة,None,None,None,سكني أو تجاري,None,None,143875,17984375
134395,منطقة القصيم,بريدة,حي الحمر,None,None,None,سكني,None,None,934,513700
134396,منطقة الرياض,الرياض,حي الملقا,None,None,None,سكني,None,None,832,2412800


In [54]:
# Save it
final_df.to_csv('./Saudi_land_prices.csv' , index = False)